In [34]:
import pandas as pd
import numpy as np
import os
from os import listdir
from os.path import isfile, join
from collections import namedtuple
from tensorflow.python.layers.core import Dense
from tensorflow.python.ops.rnn_cell_impl import _zero_state_tensors
import time
import re
from sklearn.model_selection import train_test_split
from google.colab import drive
import tensorflow as tf


In [35]:
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [36]:
def load_book(path):
    input_file = os.path.join(path)
    with open(input_file) as f:
        book = f.read()
    return book

In [37]:
path = '/content/gdrive/My Drive/DataSets/Corpus/'
book_files = [f for f in listdir(path) if isfile(join(path, f))]
print(book_files)
#book_files = book_files[1:]

['Corpus.txt']


In [38]:
books = []
for book in book_files:
    books.append(load_book(path+book))

for i in range(len(books)):
    print("Hay {} cantidad de palabras en el libro {}.".format(len(books[i].split()), book_files[i]))

books[0][:500]

Hay 16559 cantidad de palabras en el libro Corpus.txt.


'Me gusta el navegar en kayak.Los kiwis son una excelente fuente de vitamina C.\n\nPAPELES QUE A LA MUERTE DE GÜEMES\nQUEDARON EN PODER DE LA FAMILIA\n\nLa primera información, apoyada en documentos, con que\ncontamos sobre la suerte corrida por los "papeles de Güemes"\nnos la dan las cartas, hoy en nuestro poder, que Martín, el hijo\nprimogénito de Güemes, mientras acompañaba a algunos exilados\ntíos maternos en el Perú, escribió en distintas oportunidades\ndesde el Cerro de Pasco a su hermano Luis, de re'

In [39]:
def clean_text(text):
    text = re.sub(r'\n', ' ', text) 
    text = re.sub(r'[{}@_*>()\\#%+=\[\]]','', text)
    text = re.sub('a0','', text)
    text = re.sub('\'92t','\'t', text)
    text = re.sub('\'92s','\'s', text)
    text = re.sub('\'92m','\'m', text)
    text = re.sub('\'92ll','\'ll', text)
    text = re.sub('\'91','', text)
    text = re.sub('\'92','', text)
    text = re.sub('\'93','', text)
    text = re.sub('\'94','', text)
    text = re.sub('\.','. ', text)
    text = re.sub('\!','! ', text)
    text = re.sub('\?','? ', text)
    text = re.sub(' +',' ', text)
    return text

In [40]:
clean_books = []
for book in books:
    clean_books.append(clean_text(book))
    
clean_books[0][500:1500]

'reso éste en Salta desde el mismo lugar. Gracias a las del año 1847, sabemos lo siguiente: que, tras la muerte de Güemes 1821 y la de su viuda, doña Carmen Puch 1822, por minoridad de sus hijos, nacidos en 1817 y 1819 respectivamente, dichos papeles quedaron en manos de familiares cercanos; que quien después los tuvo en su poder y los ordenó fue el doctor José Redhead, el conocido médico de Güemes y de Belgrano, de gran prestigio y larga residencia en Salta “él fue el archivo de todo", según tales cartas; que, muerto el nombrado facultativo el 3 de junio de 1846, Martín Güemes y Puch, en procura de que tan valiosos documentos prestaran pronta utilidad a la Historia, pidió una y otra vez a Luis que recogiera y le remitiera "los papeles relativos a nuestro padre -decía- que debió dejar el doctor Redhead y algunas apuntaciones hechas por este sabio amigo" , para, con don Manuel Puch, fío de ambos, hacer preparar una "biografía", y, concluida que fuese ésta, el último de los nombrados los

In [41]:
vocab_to_int = {}
count = 0
for book in clean_books:
    for character in book:
        if character not in vocab_to_int:
            vocab_to_int[character] = count
            count += 1

In [42]:
codes = ['<PAD>','<EOS>','<GO>','k']
for code in codes:
    vocab_to_int[code] = count
    count += 1

In [43]:
print("El vocabulario contiene {} caracteres.".format(len(vocab_to_int)))
print(sorted(vocab_to_int))

El vocabulario contiene 101 caracteres.
['\t', ' ', '!', '"', '$', "'", ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '<EOS>', '<GO>', '<PAD>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '¡', 'ª', '«', 'º', '»', '¿', 'Á', 'É', 'Í', 'Ñ', 'Ó', 'Ú', 'Ü', 'á', 'é', 'í', 'ñ', 'ó', 'ú', 'ü', '—', '“', '”']


In [44]:
int_to_vocab = {}
for character, value in vocab_to_int.items():
    int_to_vocab[value] = character
print("El vocabulario contiene {} caracteres.".format(len(int_to_vocab)))
print(sorted(int_to_vocab))

El vocabulario contiene 101 caracteres.
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101]


In [45]:
sentences = []
for book in clean_books:
    for sentence in book.split('. '):
        sentences.append(sentence + '.')
print("Hay {} oraciones.".format(len(sentences)))

#Ejemplo
sentences[:2]


Hay 916 oraciones.


['Me gusta el navegar en kayak.',
 'Los kiwis son una excelente fuente de vitamina C.']

In [46]:
int_sentences = []

for sentence in sentences:
    int_sentence = []
    for character in sentence:
        int_sentence.append(vocab_to_int[character])
    int_sentences.append(int_sentence)

In [47]:
lengths = []
for sentence in int_sentences:
    lengths.append(len(sentence))
lengths = pd.DataFrame(lengths, columns=["counts"])
lengths.describe()

,counts
count,916.000000
mean,104.156114
std,130.711758
min,1.000000
25%,4.000000
50%,61.000000
75%,149.000000
max,1189.000000


In [48]:
max_length = 92
min_length = 10

good_sentences = []

for sentence in int_sentences:
    if len(sentence) <= max_length and len(sentence) >= min_length:
        good_sentences.append(sentence)

In [49]:
training, testing = train_test_split(good_sentences, test_size = 0.15, random_state = 2)

print("Oraciones para Entrenamiento:", len(training))
print("Oraciones para prueba:", len(testing))

Oraciones para Entrenamiento: 249
Oraciones para prueba: 44


In [50]:
training_sorted = []
testing_sorted = []

for i in range(min_length, max_length+1):
    for sentence in training:
        if len(sentence) == i:
            training_sorted.append(sentence)
    for sentence in testing:
        if len(sentence) == i:
            testing_sorted.append(sentence)

In [51]:
for i in range(5):
    print("Oracion:  " + str( training_sorted[i] ) + " Longitud: " + str( len(training_sorted[i]) ) )

Oracion:  [1, 8, 2, 43, 4, 1, 2, 30, 22, 14] Longitud: 10
Oracion:  [42, 2, 33, 1, 5, 6, 17, 9, 16, 5, 14] Longitud: 11
Oracion:  [42, 2, 1, 9, 2, 25, 16, 6, 16, 5, 48, 14] Longitud: 12
Oracion:  [23, 4, 20, 47, 16, 5, 2, 7, 51, 16, 5, 14] Longitud: 12
Oracion:  [23, 4, 20, 47, 16, 5, 2, 7, 51, 16, 5, 14] Longitud: 12


In [52]:
letters = ['a','b','c','d','e','f','g','h','i','j','k','l','m',
           'n','o','p','q','r','s','t','u','v','w','x','y','z',]

def noise_maker(sentence, threshold):
    noisy_sentence = []
    i = 0
    while i < len(sentence):
        random = np.random.uniform(0,1,1)
        if random < threshold:
            noisy_sentence.append(sentence[i])
        else:
            new_random = np.random.uniform(0,1,1)
            if new_random > 0.67:
                if i == (len(sentence) - 1):
                    continue
                else:
                    noisy_sentence.append(sentence[i+1])
                    noisy_sentence.append(sentence[i])
                    i += 1
            elif new_random < 0.33:
                random_letter = np.random.choice(letters, 1)[0]
                noisy_sentence.append(vocab_to_int[random_letter])
                noisy_sentence.append(sentence[i])
            else:
                pass     
        i += 1
    return noisy_sentence

In [53]:
threshold = 0.9
for sentence in training_sorted[:5]:
    print(sentence)
    print(noise_maker(sentence, threshold))
    print()

[1, 8, 2, 43, 4, 1, 2, 30, 22, 14]
[1, 8, 2, 43, 4, 1, 2, 30, 22, 14]

[42, 2, 33, 1, 5, 6, 17, 9, 16, 5, 14]
[2, 42, 33, 1, 5, 6, 17, 9, 7, 16, 5, 14]

[42, 2, 1, 9, 2, 25, 16, 6, 16, 5, 48, 14]
[2, 42, 1, 9, 2, 25, 16, 16, 5, 48, 14]

[23, 4, 20, 47, 16, 5, 2, 7, 51, 16, 5, 14]
[23, 4, 47, 16, 5, 2, 7, 51, 16, 5, 14]

[23, 4, 20, 47, 16, 5, 2, 7, 51, 16, 5, 14]
[23, 4, 20, 47, 16, 5, 2, 7, 16, 51, 5, 14]



In [54]:
def model_inputs():    
    with tf.compat.v1.name_scope('inputs'): #
        inputs = tf.compat.v1.placeholder(tf.int32, [None, None], name='inputs')
    with tf.compat.v1.name_scope('targets'):
        targets = tf.compat.v1.placeholder(tf.int32, [None, None], name='targets')
    keep_prob = tf.compat.v1.placeholder(tf.float32, name='keep_prob')
    inputs_length = tf.compat.v1.placeholder(tf.int32, (None,), name='inputs_length')
    targets_length = tf.compat.v1.placeholder(tf.int32, (None,), name='targets_length')
    max_target_length = tf.reduce_max(input_tensor=targets_length, name='max_target_len')

    return inputs, targets, keep_prob, inputs_length, targets_length, max_target_length

In [55]:
def process_encoding_input(targets, vocab_to_int, batch_size):
    with tf.compat.v1.name_scope("process_encoding"):
        ending = tf.strided_slice(targets, [0, 0], [batch_size, -1], [1, 1])
        dec_input = tf.concat([tf.fill([batch_size, 1], vocab_to_int['<GO>']), ending], 1)

    return dec_input

In [56]:
def encoding_layer(rnn_size, sequence_length, num_layers, rnn_inputs, keep_prob, direction): 
    if direction == 1:
        with tf.compat.v1.name_scope("RNN_Encoder_Cell_1D"):
            for layer in range(num_layers):
                with tf.compat.v1.variable_scope('encoder_{}'.format(layer)):
                    lstm = tf.compat.v1.nn.rnn_cell.LSTMCell(rnn_size)

                    drop = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                         input_keep_prob = keep_prob)

                    enc_output, enc_state = tf.compat.v1.nn.dynamic_rnn(drop, 
                                                              rnn_inputs,
                                                              sequence_length,
                                                              dtype=tf.float32)

            return enc_output, enc_state
          
    if direction == 2:
        with tf.compat.v1.name_scope("RNN_Encoder_Cell_2D"):
            for layer in range(num_layers):
                with tf.compat.v1.variable_scope('encoder_{}'.format(layer)):
                    cell_fw = tf.compat.v1.nn.rnn_cell.LSTMCell(rnn_size)
                    cell_fw = tf.contrib.rnn.DropoutWrapper(cell_fw, 
                                                            input_keep_prob = keep_prob)

                    cell_bw = tf.compat.v1.nn.rnn_cell.LSTMCell(rnn_size)
                    cell_bw = tf.contrib.rnn.DropoutWrapper(cell_bw, 
                                                            input_keep_prob = keep_prob)

                    enc_output, enc_state = tf.compat.v1.nn.bidirectional_dynamic_rnn(cell_fw, 
                                                                            cell_bw, 
                                                                            rnn_inputs,
                                                                            sequence_length,
                                                                            dtype=tf.float32)
            enc_output = tf.concat(enc_output,2)
            return enc_output, enc_state[0]

In [57]:
def training_decoding_layer(dec_embed_input, targets_length, dec_cell, initial_state, output_layer, 
                            vocab_size, max_target_length):
    with tf.compat.v1.name_scope("Training_Decoder"):
        training_helper = tf.contrib.seq2seq.TrainingHelper(inputs=dec_embed_input,
                                                            sequence_length=targets_length,
                                                            time_major=False)

        training_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                           training_helper,
                                                           initial_state,
                                                           output_layer) 

        training_logits, _ = tf.contrib.seq2seq.dynamic_decode(training_decoder,
                                                               output_time_major=False,
                                                               impute_finished=True,
                                                               maximum_iterations=max_target_length)
        return training_logits

In [58]:
def inference_decoding_layer(embeddings, start_token, end_token, dec_cell, initial_state, output_layer,
                             max_target_length, batch_size):
    with tf.compat.v1.name_scope("Inference_Decoder"):
        start_tokens = tf.tile(tf.constant([start_token], dtype=tf.int32), [batch_size], name='start_tokens')

        inference_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(embeddings,
                                                                    start_tokens,
                                                                    end_token)

        inference_decoder = tf.contrib.seq2seq.BasicDecoder(dec_cell,
                                                            inference_helper,
                                                            initial_state,
                                                            output_layer)

        inference_logits, _ = tf.contrib.seq2seq.dynamic_decode(inference_decoder,
                                                                output_time_major=False,
                                                                impute_finished=True,
                                                                maximum_iterations=max_target_length)

        return inference_logits

In [59]:
def decoding_layer(dec_embed_input, embeddings, enc_output, enc_state, vocab_size, inputs_length, targets_length, 
                   max_target_length, rnn_size, vocab_to_int, keep_prob, batch_size, num_layers, direction):   
    with tf.compat.v1.name_scope("RNN_Decoder_Cell"):
        for layer in range(num_layers):
            with tf.compat.v1.variable_scope('decoder_{}'.format(layer)):
                lstm = tf.compat.v1.nn.rnn_cell.LSTMCell(rnn_size)
                dec_cell = tf.contrib.rnn.DropoutWrapper(lstm, 
                                                         input_keep_prob = keep_prob)
    
    output_layer = Dense(vocab_size,
                         kernel_initializer = tf.compat.v1.truncated_normal_initializer(mean = 0.0, stddev=0.1))
    
    attn_mech = tf.contrib.seq2seq.BahdanauAttention(rnn_size,
                                                  enc_output,
                                                  inputs_length,
                                                  normalize=False,
                                                  name='BahdanauAttention')
    
    with tf.compat.v1.name_scope("Attention_Wrapper"):
        dec_cell = tf.contrib.seq2seq.DynamicAttentionWrapper(dec_cell,
                                                              attn_mech,
                                                              rnn_size)
    
    initial_state = tf.contrib.seq2seq.DynamicAttentionWrapperState(enc_state,
                                                                    _zero_state_tensors(rnn_size, 
                                                                                        batch_size, 
                                                                                        tf.float32))

    with tf.compat.v1.variable_scope("decode"):
        training_logits = training_decoding_layer(dec_embed_input, 
                                                  targets_length, 
                                                  dec_cell, 
                                                  initial_state,
                                                  output_layer,
                                                  vocab_size, 
                                                  max_target_length)
    with tf.compat.v1.variable_scope("decode", reuse=True):
        inference_logits = inference_decoding_layer(embeddings,  
                                                    vocab_to_int['<GO>'], 
                                                    vocab_to_int['<EOS>'],
                                                    dec_cell, 
                                                    initial_state, 
                                                    output_layer,
                                                    max_target_length,
                                                    batch_size)

    return training_logits, inference_logits

In [60]:
def seq2seq_model(inputs, targets, keep_prob, inputs_length, targets_length, max_target_length, 
                  vocab_size, rnn_size, num_layers, vocab_to_int, batch_size, embedding_size, direction):
   
    enc_embeddings = tf.Variable(tf.random.uniform([vocab_size, embedding_size], -1, 1))
    enc_embed_input = tf.nn.embedding_lookup(params=enc_embeddings, ids=inputs)
    enc_output, enc_state = encoding_layer(rnn_size, inputs_length, num_layers, 
                                           enc_embed_input, keep_prob, direction)
    
    dec_embeddings = tf.Variable(tf.random.uniform([vocab_size, embedding_size], -1, 1))
    dec_input = process_encoding_input(targets, vocab_to_int, batch_size)
    dec_embed_input = tf.nn.embedding_lookup(params=dec_embeddings, ids=dec_input)
    
    training_logits, inference_logits  = decoding_layer(dec_embed_input, 
                                                        dec_embeddings,
                                                        enc_output,
                                                        enc_state, 
                                                        vocab_size, 
                                                        inputs_length, 
                                                        targets_length, 
                                                        max_target_length,
                                                        rnn_size, 
                                                        vocab_to_int, 
                                                        keep_prob, 
                                                        batch_size,
                                                        num_layers,
                                                        direction)
    
    return training_logits, inference_logits

In [61]:
def pad_sentence_batch(sentence_batch):
    max_sentence = max([len(sentence) for sentence in sentence_batch])
    return [sentence + [vocab_to_int['<PAD>']] * (max_sentence - len(sentence)) for sentence in sentence_batch]


In [62]:
def get_batches(sentences, batch_size, threshold):

    for batch_i in range(0, len(sentences)//batch_size):
        start_i = batch_i * batch_size
        sentences_batch = sentences[start_i:start_i + batch_size]
        
        sentences_batch_noisy = []
        for sentence in sentences_batch:
            sentences_batch_noisy.append(noise_maker(sentence, threshold))
            
        sentences_batch_eos = []
        for sentence in sentences_batch:
            sentence.append(vocab_to_int['<EOS>'])
            sentences_batch_eos.append(sentence)
            
        pad_sentences_batch = np.array(pad_sentence_batch(sentences_batch_eos))
        pad_sentences_noisy_batch = np.array(pad_sentence_batch(sentences_batch_noisy))
        
        pad_sentences_lengths = []
        for sentence in pad_sentences_batch:
            pad_sentences_lengths.append(len(sentence))
        
        pad_sentences_noisy_lengths = []
        for sentence in pad_sentences_noisy_batch:
            pad_sentences_noisy_lengths.append(len(sentence))
        
        yield pad_sentences_noisy_batch, pad_sentences_batch, pad_sentences_noisy_lengths, pad_sentences_lengths


In [63]:
epochs = 100
batch_size = 128
num_layers = 2
rnn_size = 512
embedding_size = 128
learning_rate = 0.0005
direction = 2
threshold = 0.95
keep_probability = 0.75


In [64]:
def build_graph(keep_prob, rnn_size, num_layers, batch_size, learning_rate, embedding_size, direction):

    tf.compat.v1.reset_default_graph()

    inputs, targets, keep_prob, inputs_length, targets_length, max_target_length = model_inputs()

    training_logits, inference_logits = seq2seq_model(tf.reverse(inputs, [-1]),
                                                      targets, 
                                                      keep_prob,   
                                                      inputs_length,
                                                      targets_length,
                                                      max_target_length,
                                                      len(vocab_to_int)+1,
                                                      rnn_size, 
                                                      num_layers, 
                                                      vocab_to_int,
                                                      batch_size,
                                                      embedding_size,
                                                      direction)

    training_logits = tf.identity(training_logits.rnn_output, 'logits')

    with tf.compat.v1.name_scope('predictions'):
        predictions = tf.identity(inference_logits.sample_id, name='predictions')
        tf.compat.v1.summary.histogram('predictions', predictions)
    masks = tf.sequence_mask(targets_length, max_target_length, dtype=tf.float32, name='masks')
    
    with tf.compat.v1.name_scope("cost"):
        cost = tf.contrib.seq2seq.sequence_loss(training_logits, 
                                                targets, 
                                                masks)
        tf.compat.v1.summary.scalar('cost', cost)

    with tf.compat.v1.name_scope("optimze"):
        optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate)

        gradients = optimizer.compute_gradients(cost)
        capped_gradients = [(tf.clip_by_value(grad, -5., 5.), var) for grad, var in gradients if grad is not None]
        train_op = optimizer.apply_gradients(capped_gradients)

    merged = tf.compat.v1.summary.merge_all()    

    export_nodes = ['inputs', 'targets', 'keep_prob', 'cost', 'inputs_length', 'targets_length',
                    'predictions', 'merged', 'train_op','optimizer']
    Graph = namedtuple('Graph', export_nodes)
    local_dict = locals()
    graph = Graph(*[local_dict[each] for each in export_nodes])

    return graph

In [65]:
def train(model, epochs, log_string):

    with tf.compat.v1.Session() as sess:
        sess.run(tf.compat.v1.global_variables_initializer())

        testing_loss_summary = []

        iteration = 0
        
        display_step = 30 
        stop_early = 0 
        stop = 3 
        per_epoch = 3 
        testing_check = (len(training_sorted)//batch_size//per_epoch)-1

        print()
        print("Training Model: {}".format(log_string))

        train_writer = tf.compat.v1.summary.FileWriter('./logs/1/train/{}'.format(log_string), sess.graph)
        test_writer = tf.compat.v1.summary.FileWriter('./logs/1/test/{}'.format(log_string))
      
        for epoch_i in range(1, epochs+1): 
            batch_loss = 0
            batch_time = 0
            
            for batch_i, (input_batch, target_batch, input_length, target_length) in enumerate(
                    get_batches(training_sorted, batch_size, threshold)):
                start_time = time.time()
                print(start_time)
                summary, loss, _ = sess.run([model.merged,
                                             model.cost, 
                                             model.train_op], 
                                             {model.inputs: input_batch,
                                              model.targets: target_batch,
                                              model.inputs_length: input_length,
                                              model.targets_length: target_length,
                                              model.keep_prob: keep_probability})

                print(summary)
                batch_loss += loss
                end_time = time.time()
                batch_time += end_time - start_time
                train_writer.add_summary(summary, iteration)

                iteration += 1
                print(iteration)
                if batch_i % display_step == 0 and batch_i > 0:
                    print('Epoch {:>3}/{} Batch {:>4}/{} - Loss: {:>6.3f}, Seconds: {:>4.2f}'
                          .format(epoch_i,
                                  epochs, 
                                  batch_i, 
                                  len(training_sorted) // batch_size, 
                                  batch_loss / display_step, 
                                  batch_time))
                    batch_loss = 0
                    batch_time = 0

                if batch_i % testing_check == 0 and batch_i > 0:
                    batch_loss_testing = 0
                    batch_time_testing = 0
                    for batch_i, (input_batch, target_batch, input_length, target_length) in enumerate(
                            get_batches(testing_sorted, batch_size, threshold)):
                        start_time_testing = time.time()
                        summary, loss = sess.run([model.merged,
                                                  model.cost], 
                                                     {model.inputs: input_batch,
                                                      model.targets: target_batch,
                                                      model.inputs_length: input_length,
                                                      model.targets_length: target_length,
                                                      model.keep_prob: 1})

                        batch_loss_testing += loss
                        end_time_testing = time.time()
                        batch_time_testing += end_time_testing - start_time_testing

                        test_writer.add_summary(summary, iteration)

                    n_batches_testing = batch_i + 1
                    print('Testing Loss: {:>6.3f}, Seconds: {:>4.2f}'
                          .format(batch_loss_testing / n_batches_testing, 
                                  batch_time_testing))
                    
                    batch_time_testing = 0

                    testing_loss_summary.append(batch_loss_testing)
                    if batch_loss_testing <= min(testing_loss_summary):
                        print('New Record!') 
                        stop_early = 0
                        checkpoint = "./{}.ckpt".format(log_string)
                        saver = tf.compat.v1.train.Saver()
                        saver.save(sess, checkpoint)

                    else:
                        print("No Improvement.")
                        stop_early += 1
                        if stop_early == stop:
                            break

            if stop_early == stop:
                print("Stopping Training.")
                break


In [66]:
for keep_probability in [0.75]:
    for num_layers in [2]:
        for threshold in [0.95]:
            log_string = 'kp={},nl={},th={}'.format(keep_probability,
                                                    num_layers,
                                                    threshold) 
            print("true")
            model = build_graph(keep_probability, rnn_size, num_layers, batch_size, 
                                learning_rate, embedding_size, direction)
            print("true")
            train(model, epochs, log_string)


true


RuntimeError: ignored

In [67]:
def text_to_ints(text):
   
    text = clean_text(text)
    return [vocab_to_int[word] for word in text]

text = "Spellin is difficult, whch is wyh you need to study everyday."
text = text_to_ints(text)


checkpoint = "./kp=0.75,nl=2,th=0.95.ckpt"

model = build_graph(keep_probability, rnn_size, num_layers, batch_size, learning_rate, embedding_size, direction) 

with tf.compat.v1.Session() as sess:
    saver = tf.compat.v1.train.Saver()
    saver.restore(sess, checkpoint)
    
    answer_logits = sess.run(model.predictions, {model.inputs: [text]*batch_size, 
                                                 model.inputs_length: [len(text)]*batch_size,
                                                 model.targets_length: [len(text)+1], 
                                                 model.keep_prob: [1.0]})[0]

pad = vocab_to_int["<PAD>"] 

print('\nText')
print('  Word Ids:    {}'.format([i for i in text]))
print('  Input Words: {}'.format("".join([int_to_vocab[i] for i in text])))

print('\nSummary')
print('  Word Ids:       {}'.format([i for i in answer_logits if i != pad]))
print('  Response Words: {}'.format("".join([int_to_vocab[i] for i in answer_logits if i != pad])))

RuntimeError: ignored